In [20]:
import requests
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend

In [31]:
data = "Message to be timestamped"

response = requests.post('http://localhost:8090/timestamp', json={'data': data})
if response.status_code == 200:
  response_data = response.json()
  timestamp = response_data['timestamp']
  signature = bytes.fromhex(response_data['signature'])
  public_key_pem = response_data['public_key']

  public_key = serialization.load_pem_public_key(
    public_key_pem.encode(),
    backend=default_backend()
  )

  data_hash = hashes.Hash(hashes.SHA256(), backend=default_backend())
  data_hash.update(data.encode())
  digest = data_hash.finalize()

  try:
    public_key.verify(
      signature,
      digest,
      padding.PSS(
        mgf=padding.MGF1(hashes.SHA256()),
        salt_length=padding.PSS.MAX_LENGTH
      ),
      hashes.SHA256()
    )
    print(f"Signature verified for timestamp: {timestamp}")
  except Exception as e:
    print(f"Verification failed: {e}")
else:
  print("Failed to get timestamp")

Signature verified for timestamp: 2024-07-23T10:15:04.038610


In [32]:
signature

b'"U\xee\xd3U\xd8T-\x0c\x93u\xd7B\xff\xc8\xd5!\xc8K\xb8;\x86\xe7\xdd\x87\xa1\xa5\xd3\x8cs\x9eC\x19\x01\x1b\xcdQ\x97\x8a\xf7<\xcaTc\xa9B\xf1\x8d\x98^\x0bc\xae\xfb\x90\xedQ\xb2\x92\xf4\xea\xfa\xfbz\xf1=q}\xeb\x86\x0f\xd2\xd5\xe0\xad6Rt\'3\xbe\x1d1*\x10\xbdU\xbe\xc0\x80\x90\xae\xb5\x8et\xfe\x9cX^&\xdczQp\'Lg\xd54D\x05Nm\xb9\xc1\xbc\x05\x1a\xce\xd4r\x13}\x138v\x19\x9d0\xce\x91\xeb\x85\xa9\x1f\x1d\xdc\xf0*\xea=\x07n\xcb\x07\x8d\xc5\xd8\xd7U\xc1A\x15GYc\xb6|mq\x01\x15\x0b\x99\xe7\xd0\xac\x0c\xcb\xe7\xe10l\xec\x89@I\xb2\n\xf6\xd3tH\xb5\x82*D\xb5h\xfe\x05\xcd\xf6\xf4X\xd9\xa0\x82\xdf\x1a\xb9\xd7\xef\xb3\x03\xd0\\\x8a\x14\xdd\x98\x10\xcc \xf5\xc9\xde\x18W\x90\xa4 \xff\x9e\xd1)\xcf\xe1\xaa3\x88Z\xcc\xe1\xc3\xac\xd8<MC\x86\xb8\xc7\x00{\xcc\xc2\x15\x83\xd5\xaa$\x06\xa8u\x17'

In [33]:
public_key_pem

'-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAqWGPKO6eEo8hEyHGyhop\nrND4wIsJK/HYR/kE+9sT04NrWOHs21YWKATYu0ze/8JTSucTdhw7lu3uBAeiFCxa\nC6vpy+Lx9lQ/hSwcTcNWsdr/SjtjjoLqGTE5QNZrN1JtHTwevyD6HM4FbJMco2Wy\nR34eIR/A66rCzftQX/8FIMjpapfyuJ6CKfn040e6dbJfUC5IZHNVGORwzSsQ3xwf\nDc8/GXt9ETp3IZALvbcn/Xn+dinEvBMT9fGZU+Q4VMuDDZzqmi8bgvf6qZQkN1Eo\nO2hnvRMKdeoGCVCUxd/EIv02ZDHvLoMKOySHWbHB3VGVvXJyd6QPizAOdftn0vQA\nfQIDAQAB\n-----END PUBLIC KEY-----\n'